# AI-LAB SESSION 1: Uninformed search

In this first session we will work on uninformed search

## Maze environments

The environments used are **SmallMaze** (visible in the figure) and its variations
![SmallMaze](images/maze.png)
The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$

## Assignment 1

Your first assignment is to implement the BFS algorithm on **SmallMaze**. In particular, you are required to implement both *tree_search* and *graph_search* versions of BFS that will be called by the generic *bfs*.

The results returned by your *bfs* must be a tuple $(path, stats)$ in the following form:
* *path* - tuple of state identifiers forming a path from the start state to the goal state. ``None`` if no solution is found
* *stats* - tuple of:
     * *time* - time elapsed between the start and the end of the algorithm
     * *expc* - number of nodes explored. A node is considered as explored when removed from the fringe and analyzed
     * *maxnodes* - maximum number of nodes in memory at the same time (fringe + closed)

After the correctness of your implementations have been assessed, you can run the algorithms on other two maze environments: **GrdMaze** and **BlockedMaze**.

Function *build_path* can be used to return a tuple of states from the root node to another node by following *parent* links

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import gym
import envs
from timeit import default_timer as timer
from utils.fringe import FringeNode, QueueFringe, StackFringe

def build_path(node):
    
    """
    Builds a path going backward from a node
    
    Args:
        node: node to start from
        
    Returns:
        path from root to ``node``
    """
    path = []
    while node.parent is not None:
        path.append(node.state)
        node = node.parent
    return tuple(reversed(path))

The next two functions have to be implemented

In [2]:
def tree_search(env, fringe):
    """
    Tree search
    
    Args:
        environment: OpenAI Gym environment
        fringe: instance of Fringe data structure
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (expc, maxnodes): number of explored states, max nodes in memory
    """ 
    start = env.startstate
    goal = env.goalstate
    root = FringeNode(start)
    fringe.add(root)
    i = 0
    maxnodeInMem = 0
    
    while True:
        if fringe.is_empty():
            return None, (0,0)
        node = fringe.remove()
        i+=1
        if node.state == goal:
            return build_path(node), (i,len(fringe) +1)
        
        for action in range(env.action_space.n):
            stateF = env.sample(node.state, action)
            nodoF = FringeNode(stateF,node)
            fringe.add(nodoF)

In [3]:
def graph_search(env, fringe):
    """
    Graph search
    
    Args:
        environment: OpenAI Gym environment
        fringe: instance of Fringe data structure
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (expc, maxnodes): number of explored nodes, max nodes in memory
    """
    
    root = FringeNode(env.startstate)
    fringe.add(root)
    closed = []
    i = 0
    maxnodeInMem = 0
    
    while True:
        if fringe.is_empty():
            return None, (i,len(closed))
        node = fringe.remove()
        i += 1
        if node.state == env.goalstate:
            return build_path(node), (i,len(closed)+1)
        if node.state not in closed:
            closed.append(node.state)
                       
            for action in range(env.action_space.n):
                stateF = env.sample(node.state, action)
                if stateF not in closed and stateF not in fringe:
                    nodoF = FringeNode(stateF,node)
                    fringe.add(nodoF)

In [4]:
def bfs(environment, search_type):
    """
    Breadth-first search
    
    Args:
        environment: OpenAI Gym environment
        search_type: type of search - tree_search or graph_search (function pointer)
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """
    t = timer()
    path, stats = search_type(environment, QueueFringe())     
    return path, (timer() - t, stats[0], stats[1])

The following code calls your tree search version of BFS and prints the results

In [5]:
envname = "BlockedMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

print("\n----------------------------------------------------------------")
print("\tTREE SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")
##CORRETTO##

# Create and render the environment
env = gym.make(envname)

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = bfs(env, tree_search)  # Perform BFS
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	TREE SEARCH
	Environment:  BlockedMaze-v0
----------------------------------------------------------------

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'C' 'W' 'W']
 ['C' 'C' 'W' 'G']]


KeyboardInterrupt: 

Correct results for BFS tree search can be found [here](files/results/bfs_tree_search_results.txt)

The following code calls your graph search version of BFS and prints the results

In [6]:
envname = "BlockedMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

print("\n----------------------------------------------------------------")
print("\tGRAPH SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

##CORRETTO##
# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = bfs(env, graph_search)  # Perform BFS
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	GRAPH SEARCH
	Environment:  BlockedMaze-v0
----------------------------------------------------------------

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'C' 'W' 'W']
 ['C' 'C' 'W' 'G']]


Execution time: 0.0039s
N° of nodes explored: 15
Max n° of nodes in memory: 15
Solution: None


Correct results for BFS graph search can be found [here](files/results/bfs_graph_search_results.txt)

## Assignment 2

Your second assignment is to implement the IDS algorithm on **SmallMaze**. In particular, you are required to implement both *dls_ts* (depth-limited tree search) and *dls_gs* (depth-limited graph search) versions of IDS that will be called by the generic *ids*. The recursions must be implemented in *rdls_ts* (recursive depth-limited tree search) and *rdls_gs* (recursive depth-limited graph search) called by *dls_ts* and *dls_gs* respectively.

Similarly to assignment 1, the results returned by your *ids* must be a tuple $(path, stats)$ in the following form:
* *path* - tuple of state identifiers forming a path from the start state to the goal state. ``None`` if no solution is found
* *stats* - tuple of:
     * *time* - time elapsed between the start and the end of the algorithm
     * *expc* - number of nodes explored. A node is considered as explored when removed from the fringe and analyzed
     * *maxnodes* - maximum number of nodes in memory at the same time (the depth of the recursion stack + closed)

After the correctness of your implementations have been assessed, you can run the algorithms on other two maze environments: **GrdMaze** and **BlockedMaze**.

**FringeNode** has a useful variable that can be set in the constructor and can be used to track the depth of a node in the path (and consequently of the recursion stack of IDS): *pathcost*. If the root node has a *pathcost* of 0, its children will have a *pathcost* increased by 1

In [7]:
start = env.startstate
root = FringeNode(start)  # parent = None and pathcost = 0 as default
child = FringeNode(env.sample(start, 0), root, root.pathcost + 1)  # pathcost is the third argument
print("Root pathcost: {}\tChild pathcost: {}".format(root.pathcost, child.pathcost))

Root pathcost: 0	Child pathcost: 1


Here you can implement the various functions requested

In [8]:
def dls_ts(environment, limit):
    """
    Depth-limited search (tree search)
    
    Args:
        environment: OpenAI Gym environment
        limit: depth limit budget
        
    Returns:
        (path, cutoff, stats): solution as a path, cutoff flag and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """
    t = timer()
    #path, cutoff, stats = rdls_ts(environment, FringeNode(environment.startstate), limit)
    solution = rdls_ts(environment, FringeNode(environment.startstate), limit)
    return solution[0], solution[1], (timer() - t, solution[2], solution[3])

In [9]:
def rdls_ts(env, node, limit):
    """
    Recursive depth-limited search (tree search version)
    
    Args:
        environment: OpenAI Gym environment
        node: node to explore
        limit: depth limit budget
    
    Returns:
        (path, cutoff, expc, maxdepth): path, cutoff flag, number of explored nodes, max nodes in memory
    """
    if(node.state == env.goalstate):
        return (build_path(node), False, 1, limit)
    if(limit == 0):
        return (None, True, 1, limit)
    expcA = 0
    for action in range(env.action_space.n):
        childNode = FringeNode(env.sample(node.state, action), node)
        result, cutoff, expc, maxdepth = rdls_ts(env, childNode, limit - 1)
        expcA += expc
        if(not cutoff):
            return (result, False, expcA + 1, limit)
    return None, True, expcA + 1, limit

In [10]:
def dls_gs(environment, limit):
    """
    Depth-limited search (graph search)
    
    Args:
        environment: OpenAI Gym environment
        limit: depth limit budget
        
    Returns:
        (path, cutoff, stats): solution as a path, cutoff flag and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """
    
    t = timer()
    #path, cutoff, stats = rdls_ts(environment, FringeNode(environment.startstate), limit)
    solution = rdls_gs(environment, FringeNode(environment.startstate), limit, [])
    return solution[0], solution[1], (timer() - t, solution[2], solution[3])

In [11]:
def rdls_gs(env, node, limit, closed):
    """
    Recursive depth-limited search (graph search version)
    
    Args:
        environment: OpenAI Gym environment
        node: node to explore
        limit: depth limit budget
        closed: completely explored nodes
        
    Returns:
        (path, cutoff, expc, maxdepth): path, cutoff flag, n° of nodes explored, max nodes in memory
    """
    """maxdepth = 0
    expcA = 1
    if node.state not in closed:
        closed.append(node.state)
    if(node.state == env.goalstate):
        return (build_path(node), False, 1, 1)
    if(limit == 0):
        return (None, True, 1, 1)
    for action in range(env.action_space.n):
        childNode = FringeNode(env.sample(node.state, action), node)
        if childNode.state not in closed:
            #closed.append(childNode.state)
            result, cutoff, expc, maxdepth = rdls_gs(env, childNode, limit - 1,closed)
            expcA += expc
            if(not cutoff and result != None):
                return (result, False, expcA, maxdepth+1)
    return None, True, expcA, maxdepth
    """
    maxdepth = 0
    expcA = 1
    if node.state not in closed:
        closed.append(node.state)
    if node.state == env.goalstate:
        return build_path(node), False, 1, limit  # don't know how
    if limit == 0:
        return None, True, 1, limit
    cutoff_occurred = False
    #explored_nodes_counter = 1
    for action in env.actions:
        #next_state = environment.sample(node.state, action)
        childNode = FringeNode(env.sample(node.state, action), node)
        if childNode.state not in closed:
            #closed.add(next_state)
            #next_node = FringeNode(next_state, node)
            path, cutoff, expc, maxdepth = rdls_gs(env, childNode, limit-1, closed)
            expcA += expc  # 1 for this node
            if cutoff:
                cutoff_occurred = True
            elif path != None:
                return path, cutoff, expcA, maxdepth + 1
    if cutoff_occurred:
        return None, True, expcA, maxdepth + 1    
    return None, False, expcA, maxdepth

In [12]:
def ids(env, search_type):
    """
    Iterative deepening depth-first search
    
    Args:
        environment: OpenAI Gym environment
        search_type: type of search (graph or tree) - dls_gs or dls_ts (function pointer)
    
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """       
    t = timer()
    cutoff = True
    depth = 0
    expc = 0
    time = 0
    min_limit = 99999999999999999999999999999
    while(cutoff == True):
        solution, cutoff, stats = search_type(env, depth)
        #AGGIORNO STATS
        expc += stats[1]
        time = stats[0]
        if (min_limit > stats[2]):
            min_limit = stats[2]
        depth +=1
    return solution, (time, expc, depth - min_limit)

The following code calls your tree search version of IDS and prints the results

In [13]:
envname = "BlockedMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0
##CORRETTO###
print("\n----------------------------------------------------------------")
print("\tGRAPH SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = ids(env, dls_gs)  # Perform BFS
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	GRAPH SEARCH
	Environment:  BlockedMaze-v0
----------------------------------------------------------------

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'C' 'W' 'W']
 ['C' 'C' 'W' 'G']]


Execution time: 0.0034s
N° of nodes explored: 126
Max n° of nodes in memory: 12
Solution: None


In [14]:
envname = "BlockedMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

print("\n----------------------------------------------------------------")
print("\tTREE SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")
##CORRETTO##
# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = ids(env, dls_ts)  # Perform BFS
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	TREE SEARCH
	Environment:  BlockedMaze-v0
----------------------------------------------------------------

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'C' 'W' 'W']
 ['C' 'C' 'W' 'G']]


KeyboardInterrupt: 

Correct results for IDS tree search can be found [here](files/results/ids_tree_search_results.txt)

The following code calls your graph search version of BFS and prints the results

Correct results for IDS graph search can be found [here](files/results/ids_graph_search_results.txt)

## Discussion

Now that you have correctly implemented both BFS and IDS what can you say about the solutions they compute? Are there significant differences in the stats?